In [14]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [15]:
prometheus_relevancy_eval_prompt_template = """###Task Description: An instruction (might include an Input inside it), a query with response, context, and a score rubric representing evaluation criteria are given. 
       1. You are provided with evaluation task with the help of a query with response and context.
       2. Write a detailed feedback based on evaluation task and the given score rubric, not evaluating in general. 
       3. After writing a feedback, write a score that is YES or NO. You should refer to the score rubric. 
       4. The output format should look as follows: "Feedback: (write a feedback for criteria) [RESULT] (YES or NO)” 
       5. Please do not generate any other opening, closing, and explanations. 

        ###The instruction to evaluate: Your task is to evaluate if the response for the query is in line with the context information provided.

        ###Query and Response: {query_str} 

        ###Context: {context_str}
            
        ###Score Rubrics: 
        Score YES: If the response for the query is in line with the context information provided.
        Score NO: If the response for the query is not in line with the context information provided.
    
        ###Feedback: """

prometheus_relevancy_refine_prompt_template = """###Task Description: An instruction (might include an Input inside it), a query with response, context, an existing answer, and a score rubric representing a evaluation criteria are given. 
   1. You are provided with evaluation task with the help of a query with response and context and an existing answer.
   2. Write a detailed feedback based on evaluation task and the given score rubric, not evaluating in general. 
   3. After writing a feedback, write a score that is YES or NO. You should refer to the score rubric. 
   4. The output format should look as follows: "Feedback: (write a feedback for criteria) [RESULT] (YES or NO)" 
   5. Please do not generate any other opening, closing, and explanations. 

   ###The instruction to evaluate: Your task is to evaluate if the response for the query is in line with the context information provided.

   ###Query and Response: {query_str} 

   ###Context: {context_str}
            
   ###Score Rubrics: 
   Score YES: If the existing answer is already YES or If the response for the query is in line with the context information provided.
   Score NO: If the existing answer is NO and If the response for the query is in line with the context information provided.
    
   ###Feedback: """

In [16]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.evaluation import RelevancyEvaluator
from llama_index.core import Settings
from llama_index.core.schema import NodeWithScore
from typing import Any, List, Dict
from llama_index.llms.openai import OpenAI
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

class GPT4RelevancyEvaluator:
    """Class for GPT-4 Relevancy Evaluation."""
    
    def __init__(self, eval_template: str, refine_template: str):
        # Initialize GPT-4 model and relevancy evaluator
        gpt4 = OpenAI(temperature=0, model="gpt-4")
        
        
        HF_TOKEN = ""
        HF_ENDPOINT_URL = (
            ""
        )

        prometheus_llm = HuggingFaceInferenceAPI(
        model_name=HF_ENDPOINT_URL,
        token=HF_TOKEN,
        temperature=0.1,
        do_sample=True,
        top_p=0.95,
        top_k=40,
        repetition_penalty=1.1,
        )
        
        Settings.llm = prometheus_llm
        
        self.relevancy_evaluator = RelevancyEvaluator(
            eval_template=eval_template,
            refine_template=refine_template,
        )
        

    def evaluate_sources(self, query_str: str, response_vector: Any) -> List[Any]:
        """Evaluate relevancy for all source nodes in the response vector and return only 'Pass' nodes."""
        eval_source_result_full = [
            self.relevancy_evaluator.evaluate(
                query=query_str,
                response=response_vector["query_result"],
                contexts=[source_node],
            )
            for source_node in response_vector["source_node"]
        ]

        # Filter and return only the original source nodes where the evaluation passed
        pass_nodes = [
            source_node for result, source_node in zip(eval_source_result_full, response_vector["source_node"]) if result.passing
        ]
        
        return pass_nodes, len(pass_nodes)

In [17]:


from typing import Any, List, Tuple
from llama_index.core.tools.types import AsyncBaseTool
from llama_index.core.workflow import Workflow, Context, Event, StartEvent, StopEvent, step
import os
from llama_index.core.tools import FunctionTool


class FunctionCallEvent(Event):
    func_call: Tuple[str, str, str]  # Function name, raw inputs, output placeholder

class ValidateFunctionCallEvent(Event):
    input_data: List[Any]
    output_placeholder: str
    tool_output: Any

class InitializeEvent(Event):
    """Event for initializing the workflow context."""
    pass

class COAWorkFlow(Workflow):
    def __init__(self, timeout: int, max_num_sources = 3):
        super().__init__(timeout=timeout)
        self.max_num_sources = max_num_sources
        evaluator = GPT4RelevancyEvaluator(prometheus_relevancy_eval_prompt_template, prometheus_relevancy_refine_prompt_template)


        self.validator = evaluator
        
    @step(pass_context=True)
    async def initialize_step(self, ctx: Context, ev: StartEvent | InitializeEvent) -> InitializeEvent | FunctionCallEvent | StopEvent:
        """Initialize the workflow context."""
        if isinstance(ev, StartEvent):
            # Perform initial setup, e.g., load tools, validators, etc.
            ctx.data["results"] = {}
            ctx.data["tools_by_name"] = ev.tools_by_name
            ctx.data["function_calls"] = ev.func_calls
            ctx.data["iteration"] = 0
            ctx.data["accumulated_sources"] = 0

            # Produce an InitializeEvent to start initialization
            return InitializeEvent()

        # After initialization, check if there are function calls to process
        if ctx.data["function_calls"]:
            first_func_call = ctx.data["function_calls"][0]
            return FunctionCallEvent(func_call=first_func_call)
        else:
            return StopEvent({"message": "No function calls to process.", "results": ctx.data["results"]})

    @step(pass_context=True)
    async def function_call_step(self, ctx: Context, ev: FunctionCallEvent) -> ValidateFunctionCallEvent | StopEvent:
        """Execute a function call and prepare for validation."""
        func_name, raw_inputs, output_placeholder = ev.func_call
        input_data = self._prepare_inputs(ctx, raw_inputs)

        try:
            tool_output = await ctx.data["tools_by_name"][func_name].acall(*input_data)
            
            # print(f"Executed {func_name} with inputs {input_data} -> {output_placeholder}: {tool_output} \n")
        except Exception as e:
            return self._handle_error(f"Error in {func_name} with inputs {input_data}: {e}")

        
        if self.validator:
            return ValidateFunctionCallEvent(
                input_data=input_data + [tool_output.raw_output],
                output_placeholder=output_placeholder,
                tool_output=tool_output
            )


        ctx.data["results"][output_placeholder] = tool_output.content
        return await self._move_to_next_function(ctx)

    def _prepare_inputs(self, ctx: Context, raw_inputs: str) -> List[Any]:
        """Parse and prepare function inputs."""
        results = ctx.data["results"]
        input_data = []
        for raw_input in raw_inputs.split(","):
            raw_input = raw_input.strip()
            try:
                input_data.append(int(results.get(raw_input, raw_input)))
            except ValueError:
                input_data.append(raw_input)  # Handle non-integer inputs gracefully
        return input_data

    @step(pass_context=True)
    async def validate_function_step(self, ctx: Context, ev: ValidateFunctionCallEvent) -> FunctionCallEvent | StopEvent:
        """Validate function output and decide on the next step."""
        try:
            validator_output = self.validator.evaluate_sources(*ev.input_data)
            # print(f"Validation result: {validator_output} | Tool output: {ev.tool_output}")
        except Exception as e:
            return self._handle_error(f"Validation error: {e}")

        if validator_output:
            
            source_nodes, length_nodes = validator_output
            
            if length_nodes > 0:
                ctx.data["results"][ev.output_placeholder] = str(source_nodes)
                # Accumulate source if validation is successful
                ctx.data["accumulated_sources"] += length_nodes

                # Check if the maximum number of sources has been accumulated
                if ctx.data["accumulated_sources"] >= self.max_num_sources:
                    return StopEvent({"message": "Maximum number of sources accumulated.", "results": ctx.data["results"]})

            return await self._move_to_next_function(ctx)

        return self._handle_error("Tool output does not match the expected validation result")

    async def _move_to_next_function(self, ctx: Context) -> FunctionCallEvent | StopEvent:
        """Move to the next function call if available."""
        iteration = ctx.data["iteration"]
        function_calls = ctx.data["function_calls"]

        if iteration + 1 < len(function_calls):
            ctx.data["iteration"] += 1
            next_func_call = function_calls[ctx.data["iteration"]]
            return FunctionCallEvent(func_call=next_func_call)

        # End of the workflow, produce a StopEvent
        return StopEvent({"message": "All function calls processed.", "results": ctx.data["results"]})

    def _handle_error(self, message: str) -> StopEvent:
        """Handle errors gracefully and provide feedback."""
        return StopEvent({"message": message, "results": {}})


In [18]:
import nest_asyncio
import asyncio
import re
from typing import Dict, Tuple, List

from llama_index.core.tools import AsyncBaseTool, ToolOutput
from llama_index.core.types import BaseOutputParser
from llama_index.core.workflow import Context

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

class ChainOfAbstractionParser(BaseOutputParser):
    """Chain of abstraction output parser."""

    def __init__(self, verbose: bool = False):
        """Initialize the parser with verbosity and workflow setup."""
        self._verbose = verbose

    def parse(
        self, solution: str, tools_by_name: Dict[str, AsyncBaseTool]
    ) -> Tuple[str, List[ToolOutput]]:
        """Run the async parse method, handling running event loops."""
        if asyncio.get_event_loop().is_running():
            # Use `await` if inside a running event loop (Jupyter Notebook case)
            return asyncio.get_event_loop().run_until_complete(self.aparse(solution, tools_by_name))
        else:
            # Normal use case outside of Jupyter, use asyncio.run
            return asyncio.run(self.aparse(solution, tools_by_name))

    async def aparse(
        self, solution: str, tools_by_name: Dict[str, AsyncBaseTool]
    ) -> Tuple[str, List[ToolOutput]]:
        """Asynchronously parse the solution and execute the workflow."""
        # Extract function calls
        func_calls = re.findall(r"\[FUNC (\w+)\((.*?)\) = (\w+)\]", solution)
        print("THIS IS", func_calls)
        # Initialize the workflow
        workflow = COAWorkFlow(timeout=60)
        # Run the workflow
        response = await workflow.run(timeout=60,tools_by_name=tools_by_name, func_calls=func_calls)
        
        results = response["results"]
        # print(context.data)

        # Collect results from the workflow
        tool_outputs = []
        for func_name, raw_inputs, output_placeholder in func_calls:
            if output_placeholder in results:
                tool_outputs.append(
                    ToolOutput(
                        content=results[output_placeholder],
                        tool_name=func_name,
                        raw_output=results[output_placeholder],
                        raw_input={"args": raw_inputs},
                        is_error=False,
                    )
                )
            else:
                tool_outputs.append(
                    ToolOutput(
                        content="Error: No output generated",
                        tool_name=func_name,
                        raw_output=None,
                        raw_input={"args": raw_inputs},
                        is_error=True,
                    )
                )


        # Replace placeholders in the solution text
        for placeholder, value in results.items():
            solution = solution.replace(f"{placeholder}", '"' + str(value) + '"')

        return solution, tool_outputs

In [19]:
import asyncio
from typing import Dict
from pydantic import BaseModel, Field
from pdf_reader_agent.workflow import ConciergeWorkflow as DietConsultantAgent
from llama_index.core.tools import FunctionTool
from llama_index.agent.openai import OpenAIAgent

# Define a Pydantic model for input validation
class ConsultDietInput(BaseModel):
    query: str = Field(description="A question or query related to diet.")

# Asynchronous function with Pydantic validation
async def consult_diet_async(query: str) -> Dict[str, str]:
    concierge = DietConsultantAgent(timeout=180, verbose=True)
    result = await concierge.run(query=query)
    return result

# Synchronous wrapper for the asynchronous function with Pydantic validation
def consult_diet(query: str) -> Dict[str, object]:
    # Validate the input using the Pydantic model
    input_data = ConsultDietInput(query=query)
    # Run the asynchronous function
    return asyncio.run(consult_diet_async(input_data.query))

# Create the FunctionTool with the synchronous wrapper
tool = FunctionTool.from_defaults(
    fn=consult_diet,
    name="consult_diet",
    description="Consults on diet based on a query.",
)

tools_by_name = {"consult_diet": tool}



# Example solution text for diet consultation
solution = """1. Retrieve a diet recommendation by querying the diet consultant tool:
   - [FUNC consult_diet("What is the recommended diet?") = diet_result]
   2. Retrieve a rose information:
   - [FUNC consult_diet("tell me more abuout rose?") = rose_result]
"""
# Initialize parser and run the workflow
# parser = ChainOfAbstractionParser(verbose=True)
# solution, tool_outputs = await parser.aparse(solution, tools_by_name)

# tool_outputs
# print(tool_outputs)
# Print the solution and outputs
# print(solution)
# for output in tool_outputs:
#     print(output.content)b 